In [6]:
#최종 - 맥 환경용 PDF + Word 동시 생성
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os
import re
import platform
from datetime import datetime
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.shared import OxmlElement, qn

# 파일 경로 설정
EXCEL_PATH = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx"
OUTPUT_DIR = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/"

# 오늘 날짜 문자열 생성 (YYYY-MM-DD 형식)
today_str = datetime.now().strftime('%Y%m%d')
today_dir = os.path.join(OUTPUT_DIR, today_str)

# PDF와 Word 파일용 폴더 생성
pdf_dir = os.path.join(today_dir, "pdf")
doc_dir = os.path.join(today_dir, "doc")

# 폴더 생성
for dir_path in [today_dir, pdf_dir, doc_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"폴더 생성 완료: {dir_path}")

# 출력 디렉토리가 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# macOS 폰트 설정
print("macOS 폰트 검색 중...")
font_registered = False

# macOS에서 사용 가능한 한글 폰트들
possible_fonts = [
    '/System/Library/Fonts/AppleSDGothicNeo.ttc',
    '/System/Library/Fonts/AppleGothic.ttf',
    '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
    '/Library/Fonts/AppleGothic.ttf',
    '/Library/Fonts/NanumGothic.ttf',
    '/System/Library/Fonts/STHeiti Light.ttc',
    '/System/Library/Fonts/STHeiti Medium.ttc'
]

for font_path in possible_fonts:
    if os.path.exists(font_path):
        try:
            print(f"폰트 발견: {font_path}")
            if font_path.endswith('.ttc'):
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path, subfontIndex=0))
            else:
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
            font_registered = True
            print(f"폰트 등록 성공: {font_path}")
            break
        except Exception as e:
            print(f"폰트 등록 실패({font_path}): {e}")
            continue

# 폰트 등록에 실패한 경우 기본 폰트 사용
if not font_registered:
    print("경고: 한글 폰트를 등록할 수 없었습니다. 기본 폰트를 사용합니다.")
    font_name = 'Helvetica'
else:
    font_name = 'KoreanFont'

# PDF 스타일 설정
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(
    name='Normal_Justified',
    parent=styles['Normal'],
    fontName=font_name,
    alignment=TA_JUSTIFY,
    firstLineIndent=20,
    spaceBefore=12,
    spaceAfter=12,
    leading=14
))

# 엑셀 파일 로드
try:
    print(f"엑셀 파일 로드 중: {EXCEL_PATH}")
    df = pd.read_excel(EXCEL_PATH)
    print(f"총 {len(df)} 개의 행이 로드되었습니다.")
except Exception as e:
    print(f"엑셀 파일 로드 실패: {e}")
    import sys
    sys.exit(1)

# 공통 함수들
def add_hyperlink(paragraph, url, text, bold=True, font_size=14):
    """Word 문서에 하이퍼링크 추가"""
    from docx.oxml.shared import OxmlElement, qn
    
    # 하이퍼링크 관계 생성
    part = paragraph.part
    r_id = part.relate_to(url, "http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink", is_external=True)
    
    # 하이퍼링크 XML 요소 생성
    hyperlink = OxmlElement('w:hyperlink')
    hyperlink.set(qn('r:id'), r_id)
    
    # 텍스트 실행 요소 생성
    new_run = OxmlElement('w:r')
    rPr = OxmlElement('w:rPr')
    
    # 굵은 폰트 설정
    if bold:
        bold_elem = OxmlElement('w:b')
        rPr.append(bold_elem)
    
    # 폰트 크기 설정
    sz = OxmlElement('w:sz')
    sz.set(qn('w:val'), str(font_size * 2))  # Word에서는 폰트 크기를 2배로 설정
    rPr.append(sz)
    
    # 링크 스타일 설정 (파란색, 밑줄)
    color = OxmlElement('w:color')
    color.set(qn('w:val'), '0000FF')
    rPr.append(color)
    
    u = OxmlElement('w:u')
    u.set(qn('w:val'), 'single')
    rPr.append(u)
    
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)
    
    # 문단에 하이퍼링크 추가
    paragraph._p.append(hyperlink)
    
    return hyperlink

def clean_text(text):
    """HTML 태그 제거 및 텍스트 정리"""
    text = str(text)
    text = text.replace('<h3>', '').replace('</h3>', '').replace('#', '').replace('**', '')
    return text

def create_safe_filename(title):
    """파일명에 사용할 수 없는 문자 제거"""
    return re.sub(r'[\\/*?:"<>|]', '', title)

def process_body_text(body):
    """본문 텍스트 처리 - 구분선 추가 및 헤딩 처리"""
    body_lines = body.split('\n')
    processed_lines = []
    
    for line in body_lines:
        if '<h2>' in line:
            processed_lines.append("---------------------------------------------------------------------")
        processed_lines.append(line)
    
    return '\n'.join(processed_lines)

def add_line_to_word_doc(doc, text, style_type='normal'):
    """Word 문서에 텍스트 추가"""
    if style_type == 'separator':
        # 구분선 추가
        p = doc.add_paragraph(text)
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif style_type == 'heading':
        # 제목 스타일 추가
        p = doc.add_paragraph()
        run = p.add_run(text)
        run.bold = True
        run.underline = True
        run.font.size = Pt(14)
        p.space_before = Pt(16)
        p.space_after = Pt(8)
    elif style_type == 'title':
        # 메인 제목
        p = doc.add_heading(text, level=1)
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    else:
        # 일반 문단
        p = doc.add_paragraph(text)
        p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        p.paragraph_format.first_line_indent = Inches(0.28)
        p.paragraph_format.space_before = Pt(12)
        p.paragraph_format.space_after = Pt(12)

# 각 행에 대해 PDF와 Word 파일 생성
for index, row in df.iterrows():
    try:
        title = str(row['title']).strip()
        body = clean_text(row['body'])
        
        print(f"\n처리 중인 행 {index+1}/{len(df)} - 제목: {title[:30]}...")
        
        # 안전한 파일명 생성
        safe_title = create_safe_filename(title)
        
        # PDF 파일 경로
        pdf_path = os.path.join(pdf_dir, f"{safe_title}_대표 기도문 모음 나눔터.pdf")
        # Word 파일 경로
        word_path = os.path.join(doc_dir, f"{safe_title}_대표 기도문 모음 나눔터.docx")
        
        # === PDF 생성 ===
        # PDF 메타데이터를 위한 정보 준비
        from reportlab.lib.units import inch
        from reportlab.platypus import SimpleDocTemplate
        
        doc_pdf = SimpleDocTemplate(
            pdf_path,
            pagesize=A4,
            rightMargin=72,
            leftMargin=72,
            topMargin=72,
            bottomMargin=72,
            title=title,
            subject="기도문 모음",
            author="대표 기도문 나눔터",
            creator="기도문 생성기",
            keywords=f"기도문, 기도, 교회, 믿음, 신앙, {title}"
        )
        
        story = []
        
        # PDF 제목 추가
        title_style = ParagraphStyle(
            name='Title',
            parent=styles['Title'],
            fontName=font_name,
            fontSize=16,
            spaceAfter=24
        )
        
        title_safe = title.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        story.append(Paragraph(title_safe, title_style))
        story.append(Spacer(1, 0.2 * inch))
        
        # === Word 문서 생성 ===
        doc_word = Document()
        
        # Word 문서 제목 추가
        add_line_to_word_doc(doc_word, title, 'title')
        doc_word.add_paragraph()  # 빈 줄 추가
        
        # 본문 처리
        processed_body = process_body_text(body)
        
        # <h2> 태그 처리를 위한 정규 표현식
        h2_pattern = re.compile(r'<h2>(.*?)</h2>')
        
        # 구분선 스타일 정의 (PDF용)
        separator_style = ParagraphStyle(
            name='Separator',
            parent=styles['Normal'],
            fontName=font_name,
            alignment=TA_JUSTIFY
        )
        
        # 본문을 특별 태그로 변환
        styled_body = ""
        for para in processed_body.split('\n'):
            if para.strip() == "---------------------------------------------------------------------":
                styled_body += para + "\n"
                continue
                
            matches = h2_pattern.findall(para)
            if matches:
                styled_para = para
                for match in matches:
                    original = f"<h2>{match}</h2>"
                    styled_para = styled_para.replace(original, f"##HEADING_START##{match}##HEADING_END##")
                styled_body += styled_para + "\n"
            else:
                styled_body += para + "\n"
        
        # 문단 분석 및 처리
        paragraphs = []
        for para in styled_body.split('\n'):
            if para.strip() == "---------------------------------------------------------------------":
                paragraphs.append((para, "separator"))
                continue
                
            if "##HEADING_START##" in para:
                parts = para.split("##HEADING_START##")
                
                if parts[0]:
                    paragraphs.append((parts[0], "normal"))
                
                for i in range(1, len(parts)):
                    if "##HEADING_END##" in parts[i]:
                        heading_parts = parts[i].split("##HEADING_END##")
                        if heading_parts[0]:
                            paragraphs.append((heading_parts[0], "heading"))
                        if len(heading_parts) > 1 and heading_parts[1]:
                            paragraphs.append((heading_parts[1], "normal"))
            else:
                paragraphs.append((para, "normal"))
        
        # PDF와 Word 문서에 내용 추가
        for para_text, para_type in paragraphs:
            if para_text.strip() == '':
                story.append(Spacer(1, 0.1 * inch))
                continue
            
            # PDF에 추가
            if para_type == "separator":
                story.append(Paragraph(para_text, separator_style))
                add_line_to_word_doc(doc_word, para_text, 'separator')
            else:
                para_text_clean = para_text.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                para_text_clean = re.sub(r' +', ' ', para_text_clean)
                
                if para_type == "heading":
                    heading_style = ParagraphStyle(
                        name='Heading2',
                        parent=styles['Normal_Justified'],
                        fontName=font_name,
                        fontSize=14,
                        fontWeight='bold',
                        underline=True,
                        spaceBefore=16,
                        spaceAfter=8
                    )
                    story.append(Paragraph(para_text_clean, heading_style))
                    add_line_to_word_doc(doc_word, para_text, 'heading')
                else:
                    story.append(Paragraph(para_text_clean, styles['Normal_Justified']))
                    add_line_to_word_doc(doc_word, para_text, 'normal')
        
        # PDF와 Word 문서 마지막에 링크 추가
        # PDF에 링크 추가
        story.append(Spacer(1, 0.3 * inch))  # 여백 추가
        link_style = ParagraphStyle(
            name='LinkStyle',
            parent=styles['Normal'],
            fontName=font_name,
            fontSize=14,  # 폰트 크기 증가
            spaceBefore=20,
            spaceAfter=12,
            alignment=1,  # 가운데 정렬 (TA_CENTER)
            leading=20    # 줄간격 넓히기
        )
        
        # PDF 링크 텍스트 (굵은 폰트, 클릭 가능한 링크)
        link_text = '<b>더 많은 기도문 보러가기:<br/>[대표 기도문 나눔터] <link href="https://prayer-church.co.kr/" color="blue">https://prayer-church.co.kr/</link></b>'
        story.append(Paragraph(link_text, link_style))
        
        # Word 문서에 링크 추가
        doc_word.add_paragraph()  # 빈 줄 추가
        link_paragraph = doc_word.add_paragraph()
        
        # 줄간격 설정 (Word)
        from docx.shared import Pt
        link_paragraph.paragraph_format.line_spacing = Pt(20)
        
        # 첫 번째 줄 - "더 많은 기도문 보러가기:" (굵은 폰트)
        run1 = link_paragraph.add_run("더 많은 기도문 보러가기:")
        run1.bold = True
        run1.font.size = Pt(14)
        
        # 줄바꿈 추가
        link_paragraph.add_run("\n")
        
        # 두 번째 줄 - "[대표 기도문 나눔터]" (굵은 폰트)
        run2 = link_paragraph.add_run("[대표 기도문 나눔터] ")
        run2.bold = True
        run2.font.size = Pt(14)
        
        # Word에 하이퍼링크 추가 (굵은 폰트)
        try:
            # 기존 텍스트 뒤에 하이퍼링크 추가
            hyperlink_run = link_paragraph.add_run()
            hyperlink_run.bold = True
            hyperlink_run.font.size = Pt(14)
            hyperlink = add_hyperlink(link_paragraph, "https://prayer-church.co.kr/", "https://prayer-church.co.kr/")
        except:
            # 하이퍼링크 추가 실패시 일반 텍스트로 추가 (굵은 폰트)
            run3 = link_paragraph.add_run("https://prayer-church.co.kr/")
            run3.bold = True
            run3.font.size = Pt(14)
        
        link_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # PDF 파일 저장
        try:
            doc_pdf.build(story)
            print(f"PDF 생성 완료: {pdf_path}")
        except Exception as e:
            print(f"PDF 생성 실패 ({safe_title}): {e}")
        
        # Word 문서 속성 설정 (SEO 메타데이터)
        core_props = doc_word.core_properties
        core_props.title = title
        core_props.subject = "기도문 모음"
        core_props.author = "대표 기도문 나눔터"
        core_props.keywords = f"기도문, 기도, 교회, 믿음, 신앙, {title}"
        core_props.comments = f"{title} - 더 많은 기도문은 https://prayer-church.co.kr/ 에서 확인하세요"
        core_props.category = "종교/신앙"
        core_props.content_status = "최종"
        
        # Word 추가 사용자 정의 속성
        try:
            # 사용자 정의 속성 추가
            custom_props = doc_word.custom_properties
            custom_props.add("Website", "https://prayer-church.co.kr/")
            custom_props.add("Content_Type", "기도문")
            custom_props.add("SEO_Description", f"{title[:100]}... - 대표 기도문 나눔터에서 제공하는 기도문 모음")
        except Exception as e:
            print(f"사용자 정의 속성 추가 실패: {e}")
        
        # Word 파일 저장
        try:
            doc_word.save(word_path)
            print(f"Word 파일 생성 완료: {word_path}")
        except Exception as e:
            print(f"Word 파일 생성 실패 ({safe_title}): {e}")
            
    except Exception as e:
        print(f"행 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

print(f"\n모든 변환 작업이 완료되었습니다.")
print(f"PDF 파일 저장 위치: {pdf_dir}")
print(f"Word 파일 저장 위치: {doc_dir}")

macOS 폰트 검색 중...
폰트 발견: /System/Library/Fonts/AppleSDGothicNeo.ttc
폰트 등록 실패(/System/Library/Fonts/AppleSDGothicNeo.ttc): TTC file "/System/Library/Fonts/AppleSDGothicNeo.ttc": postscript outlines are not supported
폰트 발견: /System/Library/Fonts/Supplemental/AppleGothic.ttf
폰트 등록 성공: /System/Library/Fonts/Supplemental/AppleGothic.ttf
엑셀 파일 로드 중: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx
총 6 개의 행이 로드되었습니다.

처리 중인 행 1/6 - 제목: 5 월 주일 대표 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/pdf/5 월 주일 대표 기도문_대표 기도문 모음 나눔터.pdf
사용자 정의 속성 추가 실패: 'Document' object has no attribute 'custom_properties'
Word 파일 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/doc/5 월 주일 대표 기도문_대표 기도문 모음 나눔터.docx

처리 중인 행 2/6 - 제목: 5월 셋째 주일 대표 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/pdf/5월 셋째 주일 대표 기도문_대표 기도문 모음 나눔터.pdf
사용자 정의 속성 추가 실패: 'Document' object has no attribute 'custom_properties'
Word 파일 생성 완료: /Users/a/Desktop/W

In [2]:
#최종 - 맥 환경용 PDF + Word 동시 생성
import pandas as pd
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_JUSTIFY
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import os
import re
import platform
from datetime import datetime
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.shared import OxmlElement, qn

# 파일 경로 설정
EXCEL_PATH = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx"
OUTPUT_DIR = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/"

# 오늘 날짜 문자열 생성 (YYYY-MM-DD 형식)
today_str = datetime.now().strftime('%Y%m%d')
today_dir = os.path.join(OUTPUT_DIR, today_str)

# PDF와 Word 파일용 폴더 생성
pdf_dir = os.path.join(today_dir, "pdf")
doc_dir = os.path.join(today_dir, "doc")

# 폴더 생성
for dir_path in [today_dir, pdf_dir, doc_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print(f"폴더 생성 완료: {dir_path}")

# 출력 디렉토리가 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# macOS 폰트 설정
print("macOS 폰트 검색 중...")
font_registered = False

# macOS에서 사용 가능한 한글 폰트들
possible_fonts = [
    '/System/Library/Fonts/AppleSDGothicNeo.ttc',
    '/System/Library/Fonts/AppleGothic.ttf',
    '/System/Library/Fonts/Supplemental/AppleGothic.ttf',
    '/Library/Fonts/AppleGothic.ttf',
    '/Library/Fonts/NanumGothic.ttf',
    '/System/Library/Fonts/STHeiti Light.ttc',
    '/System/Library/Fonts/STHeiti Medium.ttc'
]

for font_path in possible_fonts:
    if os.path.exists(font_path):
        try:
            print(f"폰트 발견: {font_path}")
            if font_path.endswith('.ttc'):
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path, subfontIndex=0))
            else:
                pdfmetrics.registerFont(TTFont('KoreanFont', font_path))
            font_registered = True
            print(f"폰트 등록 성공: {font_path}")
            break
        except Exception as e:
            print(f"폰트 등록 실패({font_path}): {e}")
            continue

# 폰트 등록에 실패한 경우 기본 폰트 사용
if not font_registered:
    print("경고: 한글 폰트를 등록할 수 없었습니다. 기본 폰트를 사용합니다.")
    font_name = 'Helvetica'
else:
    font_name = 'KoreanFont'

# PDF 스타일 설정
styles = getSampleStyleSheet()
styles.add(ParagraphStyle(
    name='Normal_Justified',
    parent=styles['Normal'],
    fontName=font_name,
    alignment=TA_JUSTIFY,
    firstLineIndent=20,
    spaceBefore=12,
    spaceAfter=12,
    leading=14
))

# 엑셀 파일 로드
try:
    print(f"엑셀 파일 로드 중: {EXCEL_PATH}")
    df = pd.read_excel(EXCEL_PATH)
    print(f"총 {len(df)} 개의 행이 로드되었습니다.")
except Exception as e:
    print(f"엑셀 파일 로드 실패: {e}")
    import sys
    sys.exit(1)

# 공통 함수들
def clean_text(text):
    """HTML 태그 제거 및 텍스트 정리"""
    text = str(text)
    text = text.replace('<h3>', '').replace('</h3>', '').replace('#', '').replace('**', '')
    return text

def create_safe_filename(title):
    """파일명에 사용할 수 없는 문자 제거"""
    return re.sub(r'[\\/*?:"<>|]', '', title)

def process_body_text(body):
    """본문 텍스트 처리 - 구분선 추가 및 헤딩 처리"""
    body_lines = body.split('\n')
    processed_lines = []
    
    for line in body_lines:
        if '<h2>' in line:
            processed_lines.append("---------------------------------------------------------------------")
        processed_lines.append(line)
    
    return '\n'.join(processed_lines)

def add_line_to_word_doc(doc, text, style_type='normal'):
    """Word 문서에 텍스트 추가"""
    if style_type == 'separator':
        # 구분선 추가
        p = doc.add_paragraph(text)
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    elif style_type == 'heading':
        # 제목 스타일 추가
        p = doc.add_paragraph()
        run = p.add_run(text)
        run.bold = True
        run.underline = True
        run.font.size = Pt(14)
        p.space_before = Pt(16)
        p.space_after = Pt(8)
    elif style_type == 'title':
        # 메인 제목
        p = doc.add_heading(text, level=1)
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    else:
        # 일반 문단
        p = doc.add_paragraph(text)
        p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        p.paragraph_format.first_line_indent = Inches(0.28)
        p.paragraph_format.space_before = Pt(12)
        p.paragraph_format.space_after = Pt(12)

# 각 행에 대해 PDF와 Word 파일 생성
for index, row in df.iterrows():
    try:
        title = str(row['title']).strip()
        body = clean_text(row['body'])
        
        print(f"\n처리 중인 행 {index+1}/{len(df)} - 제목: {title[:30]}...")
        
        # 안전한 파일명 생성
        safe_title = create_safe_filename(title)
        
        # PDF 파일 경로
        pdf_path = os.path.join(pdf_dir, f"{safe_title}_대표 기도문 모음 나눔터.pdf")
        # Word 파일 경로
        word_path = os.path.join(doc_dir, f"{safe_title}_대표 기도문 모음 나눔터.docx")
        
        # === PDF 생성 ===
        doc_pdf = SimpleDocTemplate(
            pdf_path,
            pagesize=A4,
            rightMargin=72,
            leftMargin=72,
            topMargin=72,
            bottomMargin=72
        )
        
        story = []
        
        # PDF 제목 추가
        title_style = ParagraphStyle(
            name='Title',
            parent=styles['Title'],
            fontName=font_name,
            fontSize=16,
            spaceAfter=24
        )
        
        title_safe = title.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
        story.append(Paragraph(title_safe, title_style))
        story.append(Spacer(1, 0.2 * inch))
        
        # === Word 문서 생성 ===
        doc_word = Document()
        
        # Word 문서 제목 추가
        add_line_to_word_doc(doc_word, title, 'title')
        doc_word.add_paragraph()  # 빈 줄 추가
        
        # 본문 처리
        processed_body = process_body_text(body)
        
        # <h2> 태그 처리를 위한 정규 표현식
        h2_pattern = re.compile(r'<h2>(.*?)</h2>')
        
        # 구분선 스타일 정의 (PDF용)
        separator_style = ParagraphStyle(
            name='Separator',
            parent=styles['Normal'],
            fontName=font_name,
            alignment=TA_JUSTIFY
        )
        
        # 본문을 특별 태그로 변환
        styled_body = ""
        for para in processed_body.split('\n'):
            if para.strip() == "---------------------------------------------------------------------":
                styled_body += para + "\n"
                continue
                
            matches = h2_pattern.findall(para)
            if matches:
                styled_para = para
                for match in matches:
                    original = f"<h2>{match}</h2>"
                    styled_para = styled_para.replace(original, f"##HEADING_START##{match}##HEADING_END##")
                styled_body += styled_para + "\n"
            else:
                styled_body += para + "\n"
        
        # 문단 분석 및 처리
        paragraphs = []
        for para in styled_body.split('\n'):
            if para.strip() == "---------------------------------------------------------------------":
                paragraphs.append((para, "separator"))
                continue
                
            if "##HEADING_START##" in para:
                parts = para.split("##HEADING_START##")
                
                if parts[0]:
                    paragraphs.append((parts[0], "normal"))
                
                for i in range(1, len(parts)):
                    if "##HEADING_END##" in parts[i]:
                        heading_parts = parts[i].split("##HEADING_END##")
                        if heading_parts[0]:
                            paragraphs.append((heading_parts[0], "heading"))
                        if len(heading_parts) > 1 and heading_parts[1]:
                            paragraphs.append((heading_parts[1], "normal"))
            else:
                paragraphs.append((para, "normal"))
        
        # PDF와 Word 문서에 내용 추가
        for para_text, para_type in paragraphs:
            if para_text.strip() == '':
                story.append(Spacer(1, 0.1 * inch))
                continue
            
            # PDF에 추가
            if para_type == "separator":
                story.append(Paragraph(para_text, separator_style))
                add_line_to_word_doc(doc_word, para_text, 'separator')
            else:
                para_text_clean = para_text.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;')
                para_text_clean = re.sub(r' +', ' ', para_text_clean)
                
                if para_type == "heading":
                    heading_style = ParagraphStyle(
                        name='Heading2',
                        parent=styles['Normal_Justified'],
                        fontName=font_name,
                        fontSize=14,
                        fontWeight='bold',
                        underline=True,
                        spaceBefore=16,
                        spaceAfter=8
                    )
                    story.append(Paragraph(para_text_clean, heading_style))
                    add_line_to_word_doc(doc_word, para_text, 'heading')
                else:
                    story.append(Paragraph(para_text_clean, styles['Normal_Justified']))
                    add_line_to_word_doc(doc_word, para_text, 'normal')
        
        # PDF 파일 저장
        try:
            doc_pdf.build(story)
            print(f"PDF 생성 완료: {pdf_path}")
        except Exception as e:
            print(f"PDF 생성 실패 ({safe_title}): {e}")
        
        # Word 파일 저장
        try:
            doc_word.save(word_path)
            print(f"Word 파일 생성 완료: {word_path}")
        except Exception as e:
            print(f"Word 파일 생성 실패 ({safe_title}): {e}")
            
    except Exception as e:
        print(f"행 처리 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()

print(f"\n모든 변환 작업이 완료되었습니다.")
print(f"PDF 파일 저장 위치: {pdf_dir}")
print(f"Word 파일 저장 위치: {doc_dir}")

폴더 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/pdf
폴더 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/doc
macOS 폰트 검색 중...
폰트 발견: /System/Library/Fonts/AppleSDGothicNeo.ttc
폰트 등록 실패(/System/Library/Fonts/AppleSDGothicNeo.ttc): TTC file "/System/Library/Fonts/AppleSDGothicNeo.ttc": postscript outlines are not supported
폰트 발견: /System/Library/Fonts/Supplemental/AppleGothic.ttf
폰트 등록 성공: /System/Library/Fonts/Supplemental/AppleGothic.ttf
엑셀 파일 로드 중: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/post/post_list.xlsx
총 6 개의 행이 로드되었습니다.

처리 중인 행 1/6 - 제목: 5 월 주일 대표 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/pdf/5 월 주일 대표 기도문_대표 기도문 모음 나눔터.pdf
Word 파일 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/doc/5 월 주일 대표 기도문_대표 기도문 모음 나눔터.docx

처리 중인 행 2/6 - 제목: 5월 셋째 주일 대표 기도문...
PDF 생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/files/20250514/pdf/5월 셋째 주일 대표 기도문_대표 기도문 모음 나눔터.pdf
Word 파일 생성 완료: /Users/a/Desktop/W